In [2]:
import os
import gc
import time
import math
import datetime
from math import log, floor
from sklearn.neighbors import KDTree
import copy

import numpy as np
#import pandas as pd
from pathlib import Path
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm.notebook import tqdm as tqdm
from scipy.signal import find_peaks

import seaborn as sns
from matplotlib import colors
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

from lightgbm import LGBMRegressor
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
import multiprocessing
from functools import partial
import ray
ray.init()

import modin.pandas as pd

2022-05-18 22:22:00,505	WARNING services.py:1994 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 6709796864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.


In [ ]:
data = pd.read_csv('wtbdata_245days.csv')

In [ ]:
data = data[(data['Patv'].notnull()) & (data['Day']>160)]

In [ ]:
#lag_n = 288*7
lag_n = 288

In [ ]:
turbid = data['TurbID'].unique()
num_processes = 20

part_num = int(len(turbid)/num_processes+0.5)

df_split = []
temp_id_group = []
count = 0
for i in turbid:
    temp_id_group.append(i)
    if count%part_num==1:
        df_split.append(temp_id_group)
        temp_id_group = []
    count += 1
df_split.append(temp_id_group)

In [6]:
turbid = data['TurbID'].unique()
num_processes = 20

part_num = int(len(turbid)/num_processes+0.5)

df_split = []
temp_id_group = []
count = 0
for i in turbid:
    temp_id_group.append(i)
    if count%part_num==1:
        df_split.append(data[data['TurbID'].isin(temp_id_group)])
        temp_id_group = []
    count += 1
df_split.append(data[data['TurbID'].isin(temp_id_group)])


def lag_feature(data, feat_columns, li):    
    # 在对应的位置设置lag列
    for shift_num in li:
        start = time.time()
        temp_data = data.groupby(['TurbID'])[feat_columns].shift(shift_num)
        end1 = time.time()
        temp_data.columns=[i+"_lag_"+str(shift_num) for i in feat_columns]
        data = pd.concat([data,temp_data],axis=1)
        end2 = time.time()
        print('groupby time:', end1-start)
        print('concat time: ', end2-end1)
    return data 

def parallelize_df(df_split, func, num_processes=20):
    with multiprocessing.Pool(num_processes) as p:
        df = pd.concat(p.map(func, df_split))
    return df

In [ ]:
feat_columns = ['Wspd','Prtv','Patv']
li = [i for i in range(288,288+lag_n)]
prod = partial(lag_feature, feat_columns=feat_columns, li=li)
data_lag1 = parallelize_df(df_split, prod, 20)

In [ ]:
data_lag1

In [ ]:
data_lag1['hour'] = data_lag1['Tmstamp'].apply(lambda x:int(x.split(':')[0]))

In [ ]:
use_feat = list(['TurbID']+list(data_lag1.columns[13:]))

In [ ]:
val_data = data_lag1[data_lag1['Day'].isin([243,242])]
test_data = data_lag1[data_lag1['Day'].isin([244,245])]
train_data = data_lag1[data_lag1['Day']<242]

In [ ]:
gbm = LGBMRegressor(objective='regression',
                        num_leaves=128, 
                        learning_rate=0.07, 
                        n_estimators=1000,
                        max_depth=7)
gbm.fit(train_data[use_feat], train_data['Patv'], 
        eval_set=[(val_data[use_feat], val_data['Patv'])], 
        eval_metric='rmse', early_stopping_rounds=20)

In [ ]:
predict = gbm.predict(test_data[use_feat])

In [ ]:
test_data['predict'] = predict

In [ ]:
test_data[test_data['TurbID']==1][['Patv','predict']].plot()